In [12]:
%matplotlib
import kwant
from matplotlib import pyplot
import tinyarray
import numpy as np
from numpy import sqrt
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from numpy import exp,pi,kron,cos,sin,sqrt,pi,cosh,tanh
from pathos.multiprocessing import Pool 
from scipy import sparse, integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


sigma0 = np.array([[1,0],[0,1]])
sigmax = np.array([[0,1],[1,0]])
sigmay = np.array([[0,-1j],[1j,0]])
sigmaz = np.array([[1,0],[0,-1]])

tau0 = sigma0
taux = sigmax
tauy = sigmay
tauz = sigmaz

s0 = sigma0
sx = sigmax
sy = sigmay
sz = sigmaz


Lz =6
Lx=20
Ly=20

normal=2
Lsc=(Lz-normal)//2


M = -1.5
A1 = 2
A2 = 2
tx = 1
ty = 1
tz = 1
delta=0.5
mu=1*0


pen_len=Lz
decay_l=0.2
C0=((pen_len+(Lz//2))/decay_l)/tanh((Lz//2)/decay_l)



def kwant_square_band_TSC_G_Y(theta,phi):

    def sys_shape(pos):
        es = 1e-3
        x,z = pos
        return (-es-Lx//2 <x< Lx//2+es and -es-Lz//2< z <Lz//2+es )  
                 
    def lead_shape(pos):
        es = 1e-3
        x,z = pos
        return ( -es-Lx//2 <x< Lx//2+es and -es-Lz//2< z <Lz//2+es )  

    
    def onsite(site,ky,lam_ex,phi_sc):
        x,z = site.pos
        if z>=(Lz//2-Lsc):
            h_sc=kron(taux,kron(s0,sigma0))
            Lam_ex=0
        elif z<=-(Lz//2-Lsc):
            h_sc=cos(phi_sc)*kron(taux,kron(s0,sigma0))+sin(phi_sc)*kron(tauy,kron(s0,sigma0))
            Lam_ex=0
        else:
            h_sc=0*kron(taux,kron(s0,sigma0))
            Lam_ex=lam_ex
        onsite_e =((M+2*tx+2*ty*(1-cos(ky))+2*tz)*kron(s0,sigmaz)+A2*sin(ky)*kron(sy,sigmax)\
                   -mu*kron(s0,sigma0))

        zeeman= Lam_ex*(cos(theta)*sin(phi)*kron(sx,sigmax)+sin(theta)*sin(phi)*kron(sy,sigmax)\
                    +cos(phi)*kron(sz,sigmax))
        
        return kron(tauz,onsite_e)+ delta*h_sc+kron(tau0,zeeman)
                
    def hopping_x(site1,site2,eta):
        x1,z1 = site1.pos
        x2,z2 = site2.pos
        z=(z1+z2)/2
        hopx_e=(-tx*kron(s0,sigmaz)+A2/(2j)*kron(sx,sigmax))
        phase=eta*(z*pen_len/(Lz//2) + C0*(1/cosh(z/decay_l)**2)*tanh(z/decay_l)*\
                                              ((1/3)*(x2**3-x1**3))/decay_l)
        
#         if z1<=-(Lz//2-Lsc) or z2<=-(Lz//2-Lsc):
#             return kron(tauz,hopx_e)
#         elif z1>=(Lz//2-Lsc) or z2>=(Lz//2-Lsc):
#             return kron(tauz,hopx_e)
#         else:
        return kron(tauz,hopx_e)*cos(2*pi*phase)+1j*kron(tau0,hopx_e)*sin(2*pi*phase)
                
    def hopping_z(site1,site2,eta):
        x1,z1 = site1.pos
        x2,z2 = site2.pos
        x=(x1+x2)/2
        hopz_e=-tz*kron(s0,sigmaz)+A1/(2j)*kron(sz,sigmax)
        phase=eta*x*C0*decay_l*(tanh(z2/decay_l)-tanh(z1/decay_l))
        
#         if z1<=-(Lz//2-Lsc) or z2<=-(Lz//2-Lsc):
#             return kron(tauz,hopz_e)
#         elif z1>=(Lz//2-Lsc) or z2>=(Lz//2-Lsc):
#             return kron(tauz,hopz_e)
#         else:
        return kron(tauz,hopz_e)*cos(2*pi*phase)+1j*kron(tau0,hopz_e)*sin(2*pi*phase)

    lat = kwant.lattice.square(1,norbs=8)
    sys = kwant.Builder()
    sys[lat.shape(sys_shape, (0, 0))] = onsite
    sys[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    sys[kwant.builder.HoppingKind((0,1), lat, lat)] = hopping_z

    return sys




z=np.linspace(-Lz//2,Lz//2,201)
Ax=(1/cosh(z/decay_l))**2

# plt.figure()
# plt.plot(z, Ax,'b')
# plt.ylabel("sech$^{2}$(z/l)")
# plt.xlabel("real-space:z")
# plt.xticks([-Lz//2,-(Lz//2-Lsc),0,(Lz//2-Lsc),Lz//2])
# plt.show




"""
With potential the phase different change to 
\phi_{sc} \pm 2*pi*\phi

\phi=2B(S+2*Lx*\lambda)/(h/e)=2*\eta*B(S+2*Lx*\lambda)

\eta is the flux in unit area

"""


del_phi=0.5*pi
eta=del_phi/(2*pi*2*(Lx//2)*(Lz+2*pen_len))
Phi_scs=np.linspace(0,2,101)*pi


system_TSC_G_Y = kwant_square_band_TSC_G_Y(0,pi/2).finalized()


mod=128
Currents=np.empty((1,len(Phi_scs)),)

delta_phi=(1e-3)*pi
def cal_current_TSC_G_Y(phi_sc):
    ham_mat1 = system_TSC_G_Y.hamiltonian_submatrix(sparse=True, params = dict(ky=0.*pi,lam_ex=0,\
                                                                               eta=eta,phi_sc=phi_sc-delta_phi))
    energies1 = np.sort(sparse.linalg.eigsh(ham_mat1, k=mod,sigma=0,return_eigenvectors=False,which='LM'))

    ham_mat2 = system_TSC_G_Y.hamiltonian_submatrix(sparse=True, params = dict(ky=0.*pi,lam_ex=0,\
                                                                               eta=eta,phi_sc=phi_sc+delta_phi))
    energies2 = np.sort(sparse.linalg.eigsh(ham_mat2, k=mod,sigma=0,return_eigenvectors=False,which='LM'))

    current=np.sum((energies2[:mod//2]-energies1[:mod//2])/(2*delta_phi))

    if energies2[mod//2-1]>0:
        print("warn",energies2[mod//2-1])
    return current


with Pool() as process_pool:
    Currents[0] = process_pool.starmap_async(cal_current_TSC_G_Y,zip(Phi_scs,)).get()


plt.figure()
plt.plot(Phi_scs/pi,Currents[0]/np.max(Currents[0]),lw=3,c='k')
plt.xlabel(r"$\phi/\pi$",fontdict={'weight':'normal','size': 15})
plt.ylabel(r"$I/I_{max}$",fontdict={'weight':'normal','size': 15})
plt.xlim(0,2)
plt.xticks([0,0.5,1,1.5,2],labels=["0","0.5","1","1.5","2"],fontsize=15)
plt.yticks([-1,0,1],fontsize=15)
plt.grid(ls='--',c='gray')
plt.show




Using matplotlib backend: TkAgg


<>:142: DeprecationWarning: invalid escape sequence \p
<>:142: DeprecationWarning: invalid escape sequence \p
<ipython-input-12-3dbe5ede403d>:142: DeprecationWarning: invalid escape sequence \p
  """


<function matplotlib.pyplot.show(*args, **kw)>

In [ ]:
%matplotlib
import kwant
from matplotlib import pyplot
import tinyarray
import numpy as np
from numpy import sqrt
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from numpy import exp,pi,kron,cos,sin,sqrt,pi,cosh,tanh
from pathos.multiprocessing import Pool 
from scipy import sparse, integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


sigma0 = np.array([[1,0],[0,1]])
sigmax = np.array([[0,1],[1,0]])
sigmay = np.array([[0,-1j],[1j,0]])
sigmaz = np.array([[1,0],[0,-1]])

tau0 = sigma0
taux = sigmax
tauy = sigmay
tauz = sigmaz

s0 = sigma0
sx = sigmax
sy = sigmay
sz = sigmaz


Lz =8
Lx=30
Ly=20

normal=2
Lsc=(Lz-normal)//2


M = -1.5
A1 = 2
A2 = 2
tx = 1
ty = 1
tz = 1
delta=0.5
mu=1*0


pen_len=Lz
decay_l=0.2
C0=((pen_len+(Lz//2))/decay_l)/tanh((Lz//2)/decay_l)


def kwant_cubic_conner_TSC_Magnetic_Y(theta,phi):
    
    def sys_shape(pos):
        es = 1e-3
        x,y,z = pos
        return (  -Lx//2-es <x< Lx//2+es and -Ly//2-es <y< Ly//2+es and -es-Lz//2 < z <Lz//2+es )  

    
    def onsite(site,lam_ex,phi_sc):
        x,y,z = site.pos
        if z>=(Lz//2-Lsc):
            h_sc=kron(taux,kron(s0,sigma0))
            Lam_ex=0
        elif z<=-(Lz//2-Lsc):
            h_sc=cos(phi_sc)*kron(taux,kron(s0,sigma0))+sin(phi_sc)*kron(tauy,kron(s0,sigma0))
            Lam_ex=0
        else:
            h_sc=0*kron(taux,kron(s0,sigma0))
            Lam_ex=lam_ex
        onsite_e =((M+2*tx+2*ty+2*tz)*kron(s0,sigmaz)-mu*kron(s0,sigma0))

        zeeman= Lam_ex*(cos(theta)*sin(phi)*kron(sx,sigmaz)+sin(theta)*sin(phi)*kron(sy,sigmaz)\
                        +cos(phi)*kron(sz,sigmaz))
        
        return kron(tauz,onsite_e)+ delta*h_sc+kron(tau0,zeeman)
    
    def hopping_y(site1,site2):
        hopy_e=(-ty*kron(s0,sigmaz)+A2/(2j)*kron(sy,sigmax))
        return kron(tauz,hopy_e)
                
    def hopping_x(site1,site2,eta):
        x1,y1,z1 = site1.pos
        x2,y2,z2 = site2.pos
        z=(z1+z2)/2
        hopx_e=(-tx*kron(s0,sigmaz)+A2/(2j)*kron(sx,sigmax))
        phase=eta*(z*pen_len/(Lz//2) + C0*(1/cosh(z/decay_l)**2)*tanh(z/decay_l)*\
                                              ((1/3)*(x2**3-x1**3))/decay_l)
        
#         if z1<=-(Lz//2-Lsc) or z2<=-(Lz//2-Lsc):
#             return kron(tauz,hopx_e)
#         elif z1>=(Lz//2-Lsc) or z2>=(Lz//2-Lsc):
#             return kron(tauz,hopx_e)
#         else:
        return kron(tauz,hopx_e)*cos(2*pi*phase)+1j*kron(tau0,hopx_e)*sin(2*pi*phase)


    def hopping_z(site1,site2,eta):
        x1,y1,z1 = site1.pos
        x2,y2,z2 = site2.pos
        x=(x1+x2)/2

        hopz_e=-tz*kron(s0,sigmaz)+A1/(2j)*kron(sz,sigmax)
        phase=eta*x*C0*decay_l*(tanh(z2/decay_l)-tanh(z1/decay_l))
        
#         if z1<=-(Lz//2-Lsc) or z2<=-(Lz//2-Lsc):
#             return kron(tauz,hopz_e)
#         elif z1>=(Lz//2-Lsc) or z2>=(Lz//2-Lsc):
#             return kron(tauz,hopz_e)
#         else:
        return kron(tauz,hopz_e)*cos(2*pi*phase)+1j*kron(tau0,hopz_e)*sin(2*pi*phase)

    lat = kwant.lattice.cubic(1,norbs=8)
    sys = kwant.Builder()
    sys[lat.shape(sys_shape, (1, 1,1))] = onsite
    sys[kwant.builder.HoppingKind((1, 0,0), lat, lat)] = hopping_x
    sys[kwant.builder.HoppingKind((0,1,0), lat, lat)] = hopping_y
    sys[kwant.builder.HoppingKind((0,0,1), lat, lat)] = hopping_z

    return sys




z=np.linspace(-Lz//2,Lz//2,201)
Ax=(1/cosh(z/decay_l))**2

# plt.figure()
# plt.plot(z, Ax,'b')
# plt.ylabel("sech$^{2}$(z/l)")
# plt.xlabel("real-space:z")
# plt.xticks([-Lz//2,-(Lz//2-Lsc),0,(Lz//2-Lsc),Lz//2])
# plt.show




"""
With potential the phase different change to 
\phi_{sc} \pm 2*pi*\phi

\phi=2B(S+2*Lx*\lambda)/(h/e)=2*\eta*B(S+2*Lx*\lambda)

\eta is the flux in unit area

"""


del_phi=0.5*pi
eta=del_phi/(2*pi*2*(Lx//2)*(Lz+2*pen_len))
Phi_scs=np.linspace(0,2,101)*pi


system_TSC_G_Y = kwant_cubic_conner_TSC_Magnetic_Y(0,pi/2).finalized()


mod=128
Currents=np.empty((1,len(Phi_scs)),)

delta_phi=(1e-3)*pi
def cal_current_TSC_G_Y(phi_sc):
    ham_mat1 = system_TSC_G_Y.hamiltonian_submatrix(sparse=True, params = dict(ky=0.*pi,lam_ex=0,\
                                                                               eta=eta,phi_sc=phi_sc-delta_phi))
    energies1 = np.sort(sparse.linalg.eigsh(ham_mat1, k=mod,sigma=0,return_eigenvectors=False,which='LM'))

    ham_mat2 = system_TSC_G_Y.hamiltonian_submatrix(sparse=True, params = dict(ky=0.*pi,lam_ex=0,\
                                                                               eta=eta,phi_sc=phi_sc+delta_phi))
    energies2 = np.sort(sparse.linalg.eigsh(ham_mat2, k=mod,sigma=0,return_eigenvectors=False,which='LM'))

    current=np.sum((energies2[:mod//2]-energies1[:mod//2])/(2*delta_phi))

    if energies2[mod//2-1]>0:
        print("warn",energies2[mod//2-1])
    return current

for i in range(len(Phi_scs)):
    Currents[0,i] = cal_current_TSC_G_Y(Phi_scs[i])


plt.figure()
plt.plot(Phi_scs/pi,Currents[0]/np.max(Currents[0]),lw=3,c='k')
plt.xlabel(r"$\phi/\pi$",fontdict={'weight':'normal','size': 15})
plt.ylabel(r"$I/I_{max}$",fontdict={'weight':'normal','size': 15})
plt.xlim(0,2)
plt.xticks([0,0.5,1,1.5,2],labels=["0","0.5","1","1.5","2"],fontsize=15)
plt.yticks([-1,0,1],fontsize=15)
plt.grid(ls='--',c='gray')
plt.show




<>:142: DeprecationWarning: invalid escape sequence \p
<>:142: DeprecationWarning: invalid escape sequence \p
<ipython-input-15-efa49ec11fd4>:142: DeprecationWarning: invalid escape sequence \p
  """


Using matplotlib backend: TkAgg
